In [1]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
import torch.nn.parallel as nn
from dotenv import load_dotenv
import transformers
import os
import torch

In [2]:

print(torch.__version__) #current version
print(torch.cuda.is_available()) #check if GPU is available

2.2.1+cu121
True


In [3]:
# Check for GPU availability
if torch.cuda.is_available():
    device = torch.device("cuda")  # Move model and tensors to GPU
    print("Using GPU for computations")
else :
    print("GPU unaviable")

Using GPU for computations


In [4]:
load_dotenv()

True

In [5]:

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [6]:

tokenizer = transformers.LlamaTokenizer.from_pretrained('chaoyi-wu/PMC_LLAMA_7B')
model = transformers.LlamaForCausalLM.from_pretrained('chaoyi-wu/PMC_LLAMA_7B').to(device)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
sentence = 'i got stomach ache can you help me' 
batch = tokenizer(
            sentence,
            return_tensors="pt", 
            add_special_tokens=False
        ).to(device)

In [8]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [19]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    max_length=100,
    trust_remote_code=True,
    device="cuda" if torch.cuda.is_available() else "cpu"  # Explicit device assignment

)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [20]:
llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "What are the different types of diabetes? "

print(llm_chain.run(question))

 First, we have to know what diabetes is. 

Diabetes is a disease that affects the way your body uses glucose. 

Glucose is a type of sugar that comes from the food you eat. 

Glucose is also made by the liver. 

Glucose
